In [0]:
# Import statements
import pandas as pd
import numpy as np
import pprint
import json
from pandas.io.json import json_normalize
#import flat_table

# Data extraction

In [0]:
# Load JSON file into dataframe
data = pd.read_json('/content/drive/My Drive/frames.json')
df = pd.DataFrame(data)

In [3]:
df.head() # Looks like turns and labels are nested dictionaries

,user_id,turns,wizard_id,id,labels
0,U22HTHYNP,[{'text': 'I'd like to book a trip to Atlantis...,U21DKG18C,e2c0fc6c-2134-4891-8353-ef16d8412c9a,"{'userSurveyRating': 4.0, 'wizardSurveyTaskSuc..."
1,U21E41CQP,"[{'text': 'Hello, I am looking to book a vacat...",U21DMV0KA,4a3bfa39-2c22-42c8-8694-32b4e34415e9,"{'userSurveyRating': 3.0, 'wizardSurveyTaskSuc..."
2,U21RP4FCY,[{'text': 'Hello there i am looking to go on a...,U21E0179B,6e67ed28-e94c-4fab-96b6-68569a92682f,"{'userSurveyRating': 2.0, 'wizardSurveyTaskSuc..."
3,U22HTHYNP,[{'text': 'Hi I'd like to go to Caprica from B...,U21DKG18C,5ae76e50-5b48-4166-9f6d-67aaabd7bcaa,"{'userSurveyRating': 5.0, 'wizardSurveyTaskSuc..."
4,U21E41CQP,"[{'text': 'Hello, I am looking to book a trip ...",U21DMV0KA,24603086-bb53-431e-a0d8-1dcc63518ba9,"{'userSurveyRating': 5.0, 'wizardSurveyTaskSuc..."


In [4]:
df['id'].nunique() # There are 1369 unique ID for the dialogue

1369

In [0]:
# labels is a nested dictionary
labels_df = json_normalize(df['labels'])

In [6]:
labels_df.head()

,userSurveyRating,wizardSurveyTaskSuccessful
0,4.0,True
1,3.0,True
2,2.0,False
3,5.0,True
4,5.0,True


`userSurveyRating` - value that represents the user’s satisfaction with the Wizard’s service, ranging from 1 – complete dissatisfaction to 5 – complete satisfaction

`wizardSurveyTaskSuccessful` - boolean which is true if the wizard thinks at the end of the dialogue that the user’s goal was achieved

In [0]:
# Attempted json_normalize but kept getting the error: 'list' object has no attribute 'values'
# This happens because of list enclosing each of the dictionary.
#   - Multiple dictionaries with matching keys enclosed by a list.
#   - A dictionary with keys as columns and values in the form of list.

# Thus we try another approach
text_list = []
for item in df['turns']:
  text_list.append(item)
turns_df = pd.DataFrame(text_list)

In [158]:
turns_df.head(2) # Every row is a dialogue 

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47
0,{'text': 'I'd like to book a trip to Atlantis ...,{'db': {'result': [[{'trip': {'returning': {'d...,"{'text': 'Yes, how about going to Neverland fr...","{'db': {'result': [[], [], [], [], [], []], 's...",{'text': 'I have no flexibility for dates... b...,"{'db': {'result': [[]], 'search': [{'ORIGIN_CI...",{'text': 'I suppose I'll speak with my husband...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,"{'text': 'Hello, I am looking to book a vacati...",{'db': {'result': [[{'trip': {'returning': {'d...,{'text': 'What about a trip from Gotham City t...,{'db': {'result': [[{'trip': {'returning': {'d...,{'text': 'Would any packages to Mos Eisley be ...,{'db': {'result': [[{'trip': {'returning': {'d...,"{'text': 'You know what, I'd like to try and v...",{'db': {'result': [[{'trip': {'returning': {'d...,{'text': 'Do you have any trips from Gotham Ci...,{'db': {'result': [[{'trip': {'returning': {'d...,"{'text': 'No, that's too far for me. I need a ...",{'db': {'result': [[{'trip': {'returning': {'d...,"{'text': 'How many days would I be in Kobe?', ...","{'db': {'result': [], 'search': []}, 'text': '...",{'text': 'What would the price be if I shorten...,{'db': {'result': [[{'trip': {'returning': {'d...,"{'text': 'Ok, then I would like to purchase th...",{'db': {'result': [[{'trip': {'returning': {'d...,"{'text': 'Yes, I would like to book this packa...","{'db': {'result': [], 'search': []}, 'text': '...",,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [0]:
turns_df.replace(np.nan,'',inplace=True) # Replace all null values

In [0]:
turns_df_T = turns_df.transpose() # Transposed the dataframe to have proper conversation turns

In [0]:
# Iterates through the data frame and gets all values with a text key
convo_text = ''
for i in turns_df_T:                                 # 1369 columns
  for j in range(0,48):                              # 48 rows
    try:
      get_text = turns_df_T[i][j].get('text')        # Gets the value from the corresponding key
      convo_text += get_text + ' '                   # Concatenates all text
    except AttributeError:
      convo_text += ''                               # Concatenates '' if there is no text key

# Data Preprocessing

In [166]:
# Now we have our conversation data
print(convo_text[0:888])

I'd like to book a trip to Atlantis from Caprica on Saturday, August 13, 2016 for 8 adults. I have a tight budget of 1700. Hi...I checked a few options for you, and unfortunately, we do not currently have any trips that meet this criteria.  Would you like to book an alternate travel option? Yes, how about going to Neverland from Caprica on August 13, 2016 for 5 adults. For this trip, my budget would be 1900. I checked the availability for this date and there were no trips available.  Would you like to select some alternate dates? I have no flexibility for dates... but I can leave from Atlantis rather than Caprica. How about that? I checked the availability for that date and there were no trips available.  Would you like to select some alternate dates? I suppose I'll speak with my husband to see if we can choose other dates, and then I'll come back to you.Thanks for your help 
